##### Copyright 2019 The TensorFlow IO Authors.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Azure blob storage with TensorFlow

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/io/tutorials/azure"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/io/blob/master/docs/tutorials/azure.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/io/blob/master/docs/tutorials/azure.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
      <td>
    <a href="https://raw.githubusercontent.com/tensorflow/io/master/docs/tutorials/azure.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

## Overview

This tutorial shows how to use read and write files on [Azure Blob Storage](https://azure.microsoft.com/en-us/services/storage/blobs/) with TensorFlow, through TensorFlow IO's Azure file system integration.

An Azure account is needed to read and write files on Azure Blob Storage. However, for demo purposes, we use [Azurite](https://github.com/Azure/Azurite) which is a Azure Storage emulator in this tutorial.

## Setup and Usage

### Install required Packages, and restart runtime

In [0]:
%tensorflow_version 2.x 
!pip install tensorflow-io

### Install and Setup Azure Storage Emulator (Azurite)

In [0]:
!npm install -g azurite@2.7.0

get_ipython().system_raw('azurite-blob -s &')

### Write and Read File to Azure Storage Emulator (Azurite)

In [0]:
import os
import tensorflow as tf
import tensorflow_io as tfio

os.environ['TF_AZURE_USE_DEV_STORAGE'] = '1' # <= use Azurite emualator.

tf.io.gfile.mkdir('az://devstoreaccount1/aztest')
with tf.io.gfile.GFile("az://devstoreaccount1/aztest/hello.txt", mode='w') as w:
  w.write("Hello, world!")

with tf.io.gfile.GFile("az://devstoreaccount1/aztest/hello.txt", mode='r') as r:
  print(r.read())